In [ ]:
import qiskit
import numpy as np

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar


from PatchedMeasCal.state_prep_circuits import GHZ_prep, GHZ_state_dist


from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider()

%qiskit_job_watcher

## Setup
The idea here is that each approachs gets a maximum of 32000 measurement shots to spend
- Backend style methods will spend 50% of that on the build, 50% of that on the run
- Circuit style methods will spend 50% on their full run, and the other 50% on other circuits that need to be exectued
- AIM will divide theirs up evenly

In [ ]:
n_qubits = 7
n_meas_shots = 16000
n_build_shots = 16000


n_circuit_method_shots = n_meas_shots + n_build_shots
backend = provider.get_backend('ibm_nairobi')

n_shots_cmc = n_build_shots // (2 * len(backend.configuration().coupling_map))

circuit = GHZ_prep(backend)

err_cmap = [[0, 6], [3, 6], [4, 6], [0, 2], [1, 2], [2, 5], [2, 6]]

## Run with repetitions
If everything above is working, let's crank it out a few times

In [ ]:
n_reps = 1
results = {
    'bare':[],
    'full':[],
    'linear':[],
    'aim':[],
    'sim':[],
    'jigsaw':[],
    'cmc':[],
    'cmc_err':[]
}

for _ in range(n_reps):

    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    
    tpf_err = TensorPatchFitter(backend, n_shots=n_shots_cmc, coupling_map=err_cmap)
    tpf_err.build(verbose=True)
    
    tpf = TensorPatchFitter(backend, n_shots=n_shots_cmc)
    tpf.build(verbose=True)
    
    bare_result = bare_result_job.result().get_counts()
    
    results['bare'].append(
        GHZ_state_dist(bare_result)
    )
    
    
    results['cmc'].append(
            GHZ_state_dist(tpf.apply(bare_result))
    )
    results['cmc_err'].append(
            GHZ_state_dist(tpf_err.apply(bare_result))
    )
    results['aim'].append(GHZ_state_dist(aim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['sim'].append(GHZ_state_dist(sim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['jigsaw'].append(GHZ_state_dist(jigsaw(circuit, backend, n_circuit_method_shots, equal_shot_distribution=True)))



In [ ]:
ghz_nairobi_results = {'bare': [0.553375,
  0.5554375,
  0.55325,
  0.549875,
  0.5726875,
  0.5525625,
  0.55525,
  0.555625,
  0.5591875,
  0.5486249999999999],
 #'full': [],
 #'linear': [],
 'aim': [0.559125,
  0.5803333333333334,
  0.56375,
  0.5695416666666666,
  0.5639583333333333,
  0.5747083333333334,
  0.5677083333333334,
  0.5725,
  0.5625,
  0.5633333333333334],
 'sim': [0.6105,
  0.618875,
  0.61815625,
  0.5972187499999999,
  0.6183125,
  0.6170625,
  0.61253125,
  0.61053125,
  0.62171875,
  0.62515625],
 'jigsaw': [0.5018601596335341,
  0.656408983150848,
  0.43845249145644244,
  0.518340760339433,
  0.5518806880966112,
  0.7122806394239136,
  0.2896155744864006,
  0.36494561116402285,
  0.5395461564616844,
  0.6183232131275852],
 'cmc': [0.5809189860871635,
  0.648574695874923,
  0.6585022033086935,
  0.654537386121596,
  0.6649499218377213,
  0.6469146228332103,
  0.6440979058085586,
  0.6635841591375468,
  0.6510904750165334,
  0.6337076949513196],
 'cmc_err': [0.249237489158588,
  0.2335192527421946,
  0.21365042967272457,
  0.21752806216764553,
  0.25360882077965474,
  0.7014351090443616,
  0.2097346032092665,
  0.24846343803275317,
  0.7339895705059154,
  0.21675982518896447]}

In [ ]:
res = ghz_nairobi_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')

In [ ]:
for i in res:
    print(i, np.max(res[i]))